In [ ]:
# Use Eurlex collection from http://bigartm.readthedocs.io/en/master/download.html
# Take one batch and save it without @label_class
import artm
batch = artm.messages.Batch()
batch_name = r'C:\datasets\eurlex-batches\3b055c18-206b-42cf-84d7-d20e2095bdb6.batch'
with open(batch_name, "rb") as f:
    batch.ParseFromString(f.read())

print set(batch.class_id)

batch2 = artm.messages.Batch()
batch2.token.extend(batch.token)
batch2.class_id.extend(batch.class_id)
batch2.description = batch.description
batch2.id = u'3b055c18-206b-42cf-84d7-d20e2095bdb6'

class_id = '@default_class'
for item in batch.item:
    item2 = batch2.item.add()
    item2.id = item.id
    item2.title = item.title
    for field in item.field:
        for token_id, token_count in zip(field.token_id, field.token_count):
            if batch.class_id[token_id] == class_id:
                item2.token_id.append(token_id)
                item2.token_weight.append(float(token_count))

with open (r'C:\datasets\eurlex-batches-test-dflt\3b055c18-206b-42cf-84d7-d20e2095bdb6.batch', "wb") as f:
    f.write(batch2.SerializeToString())
    
print 'Done'

In [62]:
import artm
batch_vectorizer = artm.BatchVectorizer(data_path=r'C:\datasets\eurlex-batches',
                             data_format='batches')

model = artm.ARTM(num_topics=15,
                  class_ids={'@labels_class':1, '@default_class':1},
                  num_document_passes=3,
                  dictionary=batch_vectorizer.dictionary,
                  scores=[artm.PerplexityScore(name='s1')])

model.fit_offline(batch_vectorizer=batch_vectorizer,
                  num_collection_passes=3)

print 'perplexity: {}'.format(model.score_tracker['s1'].value)

perplexity: [19240.585963931935, 1574.3602987346392, 1549.2087565126892]


In [64]:
batch_vectorizer1 = artm.BatchVectorizer(data_path=r'C:\datasets\eurlex-batches-test-both', data_format='batches')
batch_vectorizer2 = artm.BatchVectorizer(data_path=r'C:\datasets\eurlex-batches-test-dflt', data_format='batches')

model.class_ids={'@default_class':1, '@labels_class':1}
print 'Use two modalities                     : {}'.format(model.transform(batch_vectorizer1).iloc[0,0])
print 'Use @def modality (separate batch)     : {}'.format(model.transform(batch_vectorizer2).iloc[0,0])

model.class_ids={'@default_class':1}
print 'Use @def modality (via model.class_ids): {}'.format(model.transform(batch_vectorizer1).iloc[0,0])

Use two modalities                     : 0.0394030660391
Use @def modality (separate batch)     : 0.0392328128219
Use @def modality (via model.class_ids): 0.0392328128219
